# **CH02.3. Attention**

#### **`Paper Info`** : Effective Approaches to Attention-based Neural Machine Translation  
#### $ \hspace{1.75cm} - $ Thang Luong, Hieu Pham, Christopher D. Manning et al. (EMNLP/2015)

<b></b>

> ## **요약(Summary)**

| Item | Description |
|------|------|
| Research topic | Attention-based NMT에서 Global/Local attention 및 Input-feeding 구조의 효과적 설계와 정렬 함수 비교 |
| Core idea | Global은 전체 소스에 soft alignment를 적용하고 Local은 예측 중심 $ p_{t} $ 인근 window만 보며 Gaussian prior를 곱해 연산을 절감하고 정렬을 안정화함 |
| Key findings | $ \cdot{} $ Local-p가 연산 효율과 성능을 양립 <br> $ \cdot{} $ Input-feeding으로 누적 정렬 정보를 디코더에 주입하여 BLEU 향상 <br> $ \cdot{} $ 길이가 긴 문장에서 어텐션 모델이 강건함을 길이 분석으로 확인 |
| Contributions | $ \cdot{} $ Global/Local attention의 통일적 정식화와 정렬 함수(dot/general/concat/location) 비교 <br> $ \cdot{} $ Predictive Local attention과 Input-feeding 제안 <br> $ \cdot{} $ WMT En↔De에서 당시 SOTA 달성 및 정량·정성 분석 제시 |


<b></b>

> ## **연구 배경(Motivation & Prior Work)**

#### **(1) 기존 연구 및 한계점** :
##### $ \hspace{0.15cm} \cdot{} $ RNN encoder–decoder의 고정 길이 context vector는 긴 문장에서 정보 병목이 발생함
##### $ \hspace{0.15cm} \cdot{} $ Bahdanau attention은 정렬을 학습해 병목을 완화했지만 계산 경로가 복잡하고 구조적 설계 공간 탐색이 제한적이었음
##### $ \hspace{0.15cm} \cdot{} $ Attention의 scope와 정렬 함수 선택, 그리고 정렬 정보를 디코더에 어떻게 누적 반영할지에 대한 체계적 비교가 부족했음
#### **(2) 연구 목표** :
##### $ \hspace{0.15cm} \cdot{} $ Attention의 설계 선택지(Global vs Local, 정렬 함수, 정렬 누적 방식)가 번역 품질과 효율에 미치는 영향을 정량적으로 규명
##### $ \hspace{0.15cm} \cdot{} $ 긴 문장에서도 견고한 성능을 보이는 간결하고 효율적인 attention 아키텍처 확립
##### $ \hspace{0.15cm} \cdot{} $ 표준 지표(BLEU, PPL)와 길이별 분석으로 효과를 실증
#### **(3) 제안된 방법론** :
##### $ \hspace{0.15cm} \cdot{} $ Global attention으로 전체 소스 은닉을 가중 평균하여 정보 손실을 최소화
##### $ \hspace{0.15cm} \cdot{} $ Local attention으로 예측 중심 $ p_{t} $ 주변의 제한된 window만 주의하여 계산량을 감소
##### $ \hspace{0.15cm} \cdot{} $ Input-feeding으로 직전 시점의 어텐션 결합 상태를 다음 시점 입력에 주입해 정렬 누적 효과를 모델링

<b></b>

> ## **방법론(Method)**

#### **(1) Notation & 기본식** :
##### $ \hspace{0.15cm} \cdot{} $ 소스 토큰열 $ x = (x_{1}, \ldots{}, x_{S}) $, 타깃 토큰열 $ y = (y_{1}, \ldots{}, y_{T}) $
##### $ \hspace{0.15cm} \cdot{} $ 인코더 상위 은닉 $ \bar{h}_{s} \in \mathbb{R}^{d_{h}} $, 디코더 상위 은닉 $ h_{t} \in \mathbb{R}^{d_{h}} $
##### $ \hspace{0.15cm} \cdot{} $ 조건부 확률 분해와 학습 목표는 최대우도 추정으로 설정
##### $ \hspace{0.15cm} \Rightarrow{} \log{} p(y \mid{} x) = \sum_{t=1}^{T} \log{} p(y_{t} \mid{} y_{<t}, x) \;\; \text{ where } \, y_{<t} = (y_{1},\ldots{},y_{t-1}) $
##### $ \hspace{0.15cm} \Rightarrow{} J(\theta{}) = - \sum\nolimits_{(x,y)} \log{} p(y \mid{} x;\theta{}) $
##### $ \hspace{0.15cm} \cdot{} $ 어텐션 결합 상태로 다음 단어 분포를 산출
##### $ \hspace{0.15cm} \Rightarrow{} \tilde{h}_{t} = \tanh{}\!\big(W_{c}[c_{t};h_{t}]\big), \;\; p(y_{t}\mid y_{<t},x)=\mathrm{softmax}(W_{s}\tilde{h}_{t}) \;\; \text{ where } \, c_{t} \in \mathbb{R}^{d_{h}} $
#### **(2) Global attention** :
##### $ \hspace{0.15cm} \cdot{} $ 모든 소스 위치에 대한 내용 기반 정렬 가중치로 문맥 $ c_{t} $를 구성
##### $ \hspace{0.15cm} \Rightarrow{} a_{t}(s) = \frac{\exp{}\{\mathrm{score}(h_{t}, \bar{h}_{s})\}}{\sum\nolimits_{s'} \exp{}\{\mathrm{score}(h_{t}, \bar{h}_{s'})\}} , \;\; c_{t} = \sum\nolimits_{s} a_{t}(s)\bar{h}_{s} $
##### $ \hspace{0.15cm} \cdot{} $ 정렬 함수 후보
##### $ \hspace{0.15cm} \Rightarrow{} \mathrm{dot}: \, \mathrm{score} = h_{t}^{\top} \bar{h}_{s} \;\; \text{ where } \, h_{t}, \bar{h}_{s} \in \mathbb{R}^{d_{h}} $
##### $ \hspace{0.15cm} \Rightarrow{} \mathrm{general}: \, \mathrm{score} = h_{t}^{\top} W_{a} \bar{h}_{s} \;\; \text{ where } \, W_{a} \in \mathbb{R}^{d_{h}\times d_{h}} $
##### $ \hspace{0.15cm} \Rightarrow{} \mathrm{concat}: \, \mathrm{score} = v_{a}^{\top} \tanh{}\!\big(W_{a}[h_{t};\bar{h}_{s}]\big) \;\; \text{ where } \, v_{a} \in \mathbb{R}^{d_{h}} $
##### $ \hspace{0.15cm} \Rightarrow{} \mathrm{location}: \, a_{t} = \mathrm{softmax}(W_{a} h_{t}) \;\; \text{ where } \, a_{t} \in \mathbb{R}^{S} \text{ after length-wise trim} $
##### **(`PLUS`) 해석** :
##### $ \hspace{0.15cm} \cdot{} $ Global은 정보 손실이 없으나 매 시점 $ O(S) $ 정렬 계산을 수행
#### **(3) Local attention** :
##### $ \hspace{0.15cm} \cdot{} $ 중심 $ p_{t} $ 주변 $ [p_{t}-D, \, p_{t}+D] $ window에서만 정렬과 집계를 수행
##### $ \hspace{0.15cm} \cdot{} $ Local-m(monotonic): $ p_{t} = t $ 가정으로 단조 정렬을 근사
##### $ \hspace{0.15cm} \cdot{} $ Local-p(predictive): $ p_{t} $를 연속적으로 예측하고 Gaussian prior를 곱해 근접 위치를 선호
##### $ \hspace{0.15cm} \Rightarrow{} p_{t} = S \cdot \sigma{}\!\big(v_{p}^{\top} \tanh{}\!(W_{p} h_{t})\big) \;\; \text{ where } \, v_{p} \in \mathbb{R}^{d_{h}}, \, W_{p} \in \mathbb{R}^{d_{h}\times d_{h}} $
##### $ \hspace{0.15cm} \Rightarrow{} a_{t}(s) \propto \exp{}\{\mathrm{score}(h_{t},\bar{h}_{s})\} \cdot \exp{}\!\Big(- \frac{(s-p_{t})^{2}}{2\sigma{}^{2}}\Big), \; \sigma{} = D/2 $
##### $ \hspace{0.15cm} \cdot{} $ 마지막에 window 내에서 softmax 정규화하여 $ \sum\nolimits_{s} a_{t}(s) = 1 $ 보장
##### **(`PLUS`) 해석** :
##### $ \hspace{0.15cm} \cdot{} $ Local은 매 시점 $ O(D) $로 계산을 절감하면서 $ p_{t} $ 예측을 통해 비단조 정렬도 유연하게 처리
#### **(4) Input-feeding** :
##### $ \hspace{0.15cm} \cdot{} $ 직전 어텐션 결합 은닉 $ \tilde{h}_{t} $를 다음 시점 입력과 결합해 정렬 결정을 누적 반영
##### $ \hspace{0.15cm} \Rightarrow{} h_{t+1} = f\!\big(h_{t}, \, [\mathrm{Emb}(y_{t}); \tilde{h}_{t}]\big) \;\; \text{ where } \, f:\text{LSTM/GRU update}, \, \tilde{h}_{t}=\tanh{}\!(W_{c}[c_{t};h_{t}]) $
##### **(`PLUS`) 해석** :
##### $ \hspace{0.15cm} \cdot{} $ 이미 번역된 소스 토큰 재방문을 억제하고 누락을 줄이는 coverage-like 효과를 제공


<b></b>

> ## **실험(Experiments)**

#### **(1) 실험 설정** :
##### $ \hspace{0.15cm} \cdot{} $ 데이터셋
| Item | Description |
|------|--------------|
| Training dataset | WMT14 En–De, 4.5M sent pairs, 116M En, 110M De |
| Dev / Test sets | newstest2013 / newstest2014, newstest2015 |
| Max length | 50 tokens filtered |
| Vocabulary | 50K most frequent |
| Preprocess | Shuffle mini-batches, source reversing |
##### $ \hspace{0.15cm} \cdot{} $ 모델 및 훈련
| Item | Description |
|------|--------------|
| Encoder / Decoder | Stacked LSTM, 4 layers |
| Hidden size | 1000 |
| Embedding dim | 1000 |
| Attention window | Local: $ D = 10 $ |
| Optimizer | SGD, LR decay (halving) |
| Batch size | 128 |
| Dropout | 0.2 |
| Training time | ? |
| Hardware | ? |
| Metrics | BLEU, PPL |
| Baselines | Non-attentional seq2seq, Bahdanau-style |
#### **(2) 분석 및 결과** :
##### $ \hspace{0.15cm} \cdot{} $ Global(dot)과 Local-p가 전반적으로 강한 성능을 보였고 Input-feeding이 추가 향상을 제공
##### $ \hspace{0.15cm} \cdot{} $ UNK-replacement와의 결합으로 추가 BLEU 향상이 관찰
##### $ \hspace{0.15cm} \cdot{} $ Length analysis에서 긴 문장 길이 구간에서도 성능 저하가 완만하여 attention 구조의 강건성이 확인
#### **(3) 결과 해석** :
##### $ \hspace{0.15cm} \cdot{} $ Global은 정보 손실이 없으나 길이가 증가할수록 연산 비용이 커지는 반면 Local-p는 예측 중심과 Gaussian prior로 효율성과 품질을 양립
##### $ \hspace{0.15cm} \cdot{} $ Input-feeding은 누적 정렬 정보를 전달하여 반복 번역과 누락을 줄여 정량 지표의 일관된 개선을 유도

<b></b>

> ## **결론(Conclusion)**

#### **(1)** Global/Local attention과 Input-feeding의 간결한 조합이 NMT 품질과 효율을 동시에 개선
#### **(2)** Predictive Local attention은 길이가 긴 문장에서도 견고하며 계산량을 실용 범위로 유지
#### **(3)** 정렬 함수 선택(dot/general/concat/location)과 정렬 누적 전략이 실제 성능을 좌우하는 핵심 설계 축임

<b></b>